# Params Init with: string of other var, function string

In [1]:
from brian2 import *
from brian2tools import *

prefs.codegen.target = 'cython'  # weave is not multiprocess-safe!

WARNING    /home/alteska/anaconda3/envs/brian_opt/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
 [py.warnings]


In [2]:
# Parameters
area = 20000*umetre**2
Cm = 1*ufarad*cm**-2 * area
El = -65*mV
EK = -90*mV
ENa = 50*mV
VT = -63*mV

dt = 0.01*ms

In [3]:
defaultclock.dt = dt

In [4]:
# Generate a step-current input and an "experimental" voltage trace
input_current = np.hstack([np.zeros(int(5*ms/dt)), np.ones(int(5*ms/dt)), np.zeros(int(5*ms/dt))])*nA

params_correct = {'gl': float(5e-5*siemens*cm**-2 * area),
                  'g_na': float(100*msiemens*cm**-2 * area),
                  'g_kd': float(30*msiemens*cm**-2 * area)}

In [13]:
# The model
eqsHH = Equations('''
dv/dt = (gl*(El-v) - g_na*(m*m*m)*h*(v-ENa) - g_kd*(n*n*n*n)*(v-EK) + I(t))/Cm : volt
dm/dt = 0.32*(mV**-1)*(13.*mV-v+VT)/
    (exp((13.*mV-v+VT)/(4.*mV))-1.)/ms*(1-m)-0.28*(mV**-1)*(v-VT-40.*mV)/
    (exp((v-VT-40.*mV)/(5.*mV))-1.)/ms*m : 1
dn/dt = 0.032*(mV**-1)*(15.*mV-v+VT)/
    (exp((15.*mV-v+VT)/(5.*mV))-1.)/ms*(1.-n)-.5*exp((10.*mV-v+VT)/(40.*mV))/ms*n : 1
dh/dt = 0.128*exp((17.*mV-v+VT)/(18.*mV))/ms*(1.-h)-4./(1+exp((40.*mV-v+VT)/(5.*mV)))/ms*h : 1
g_na : siemens (constant)
g_kd : siemens (constant)
gl   : siemens (constant)
''',
Cm=1*ufarad*cm**-2 * area, El=-65*mV, EK=-90*mV, ENa=50*mV, VT=-63*mV)

In [33]:
I = TimedArray(input_current, dt=dt)

G = NeuronGroup(1, eqsHH, method='exponential_euler', namespace=None)
G.v = Elg_kd : siemens (constant)
gl   : siemens (constant)
Cm   : farad (constant)
G.set_states(params_correct, units=False)
mon = StateMonitor(G, 'v', record=0)
run(25*ms)

voltage = mon.v[0]/mV
voltage += np.random.randn(len(voltage))

In [34]:
inp_trace = np.array([input_current])
n0, n1 = inp_trace.shape

In [35]:
out_trace = np.array(voltage[:n1])

In [36]:
G.get_states()

{'N': array(1),
 'i': array([0], dtype=int32),
 't': 25. * msecond,
 'dt': 10. * usecond,
 't_in_timesteps': array(2500),
 'v': array([-72.35369668]) * mvolt,
 'm': array([0.00192739]),
 'n': array([0.00677638]),
 'h': array([0.99967374]),
 'g_na': array([20.]) * usiemens,
 'g_kd': array([6.]) * usiemens,
 'gl': array([10.]) * nsiemens}

In [37]:
G.namespace['El']

KeyError: 'El'

In [28]:
G.set_states({'El': -70*mV})

KeyError: 'El'

In [20]:
G.namespace['El']

KeyError: 'El'

## Model Fitting 

In [10]:
# Model for modelfitting
eqs = Equations(
'''
dv/dt = (gl*(El-v) - g_na*(m*m*m)*h*(v-ENa) - g_kd*(n*n*n*n)*(v-EK) + I)/Cm : volt
dm/dt = 0.32*(mV**-1)*(13.*mV-v+VT)/
    (exp((13.*mV-v+VT)/(4.*mV))-1.)/ms*(1-m)-0.28*(mV**-1)*(v-VT-40.*mV)/
    (exp((v-VT-40.*mV)/(5.*mV))-1.)/ms*m : 1
dn/dt = 0.032*(mV**-1)*(15.*mV-v+VT)/
    (exp((15.*mV-v+VT)/(5.*mV))-1.)/ms*(1.-n)-.5*exp((10.*mV-v+VT)/(40.*mV))/ms*n : 1
dh/dt = 0.128*exp((17.*mV-v+VT)/(18.*mV))/ms*(1.-h)-4./(1+exp((40.*mV-v+VT)/(5.*mV)))/ms*h : 1
g_na : siemens (constant)
g_kd : siemens (constant)
gl   : siemens (constant)
''',
Cm=1*ufarad*cm**-2 * area, El=-65*mV, EK=-90*mV, ENa=50*mV, VT=-63*mV)

In [11]:
n_opt = NevergradOptimizer()
metric = MSEMetric()

In [12]:
# pass parameters to the NeuronGroup
res, error = fit_traces(model=eqs, input_var='I', output_var='v',
                        input=inp_trace * amp, output=[out_trace]*mV, dt=dt,
                        param_init={'v': '-65*mV'},
                        method='exponential_euler',
                        callback=None,
                        gl=[1e-8*siemens*cm**-2 * area, 1e-4*siemens*cm**-2 * area],
                        g_na=[1*msiemens*cm**-2 * area, 200*msiemens*cm**-2 * area],
                        g_kd=[1*msiemens*cm**-2 * area, 100*msiemens*cm**-2 * area],
                        n_rounds=2, n_samples=5, optimizer=n_opt, metric=metric
                        )

{'N': array(5), 'i': array([0, 1, 2, 3, 4], dtype=int32), 't': 25. * msecond, 'dt': 10. * usecond, 't_in_timesteps': array(2500), 'v': array([0., 0., 0., 0., 0.]) * volt, 'm': array([0., 0., 0., 0., 0.]), 'n': array([0., 0., 0., 0., 0.]), 'h': array([0., 0., 0., 0., 0.]), 'g_na': array([0., 0., 0., 0., 0.]) * siemens, 'g_kd': array([0., 0., 0., 0., 0.]) * siemens, 'gl': array([0., 0., 0., 0., 0.]) * siemens}


KeyError: 'El'

In [ ]:
# give information to the optimizer
print('correct:', params_correct, '\n output:', res)
print('error', error)

In [ ]:
# visualization of the results
start_scope()

fits = generate_fits(model=eqs, method='exponential_euler', params=res,
                     input=inp_trace * amp, input_var='I', output_var='v',
                     dt=dt, param_init={'v': -65*mV})

In [ ]:
fig, ax = plt.subplots(nrows=1)
ax.plot(out_trace)
ax.plot(fits[0]/mV)
plt.title('nevergrad optimization')
plt.show()